In [257]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score,GridSearchCV

In [135]:
df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
df.head(16)

,Offered_By,Category,Rating,Reviews,Size,Price,Content_Rating,Last_Updated_On,Release_Version,OS_Version_Required,Downloads
0,ps_id-24654,Finance,4.18,1481,Varies with device,Free,Everyone,May 05 2020,Varies with device,Varies with device,"100,000+"
1,ps_id-35329,Music And Audio,4.81,302,10M,Free,Everyone,Mar 26 2020,3.9.18,4.1 and up,"5,000+"
2,ps_id-11044,Game Casual,4.27,374,27M,Free,Everyone,May 01 2020,1.10.1,4.1 and up,"10,000+"
3,ps_id-36068,Business,4.03,122058,Varies with device,Free,Teen,May 02 2020,Varies with device,Varies with device,"10,000,000+"
4,ps_id-35831,Medical,4.60,358,Varies with device,297.5742,Everyone,Nov 29 2018,Varies with device,Varies with device,"5,000+"
5,ps_id-17918,Books And Reference,4.80,40,2.3M,Free,Everyone,Feb 05 2019,3,4.1 and up,"10,000+"
6,ps_id-11727,Finance,4.60,1992,24M,Free,Everyone,Jul 25 2019,3000.5.3532,5.0 and up,"50,000+"
7,ps_id-4496,Tools,4.62,37975,20M,Free,Everyone,May 05 2020,1.28.1,4.4 and up,"1,000,000+"
8,ps_id-9590,Finance,3.20,191,10M,Free,Everyone,Mar 01 2019,4.5.6,4.0 and up,"10,000+"
9,ps_id-7167,Game,4.10,349503,19M,Free,Everyone,Sep 29 2016,1.31,4.0 and up,"10,000,000+"


In [136]:
df['Downloads'].unique()

array(['100,000+', '5,000+', '10,000+', '10,000,000+', '50,000+',
       '1,000,000+', '5,000,000+', '500,000+', '1,000+', '1,000,000,000+',
       '50,000,000+', '100+', '100,000,000+', '500+', '10+', '50+',
       '500,000,000+', '5,000,000,000+'], dtype=object)

In [137]:
df['Downloads'] = df['Downloads'].replace(['10+','50+','100+','500+','1,000+','5,000+','10,000+','50,000+','100,000+','500,000+','1,000,000+','5,000,000+','10,000,000+','50,000,000+','100,000,000+','500,000,000+','1,000,000,000+','5,000,000,000+'],[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
df['Downloads'] = df['Downloads'].astype(int)
df.isnull().sum(axis=0)

Offered_By             0
Category               0
Rating                 0
Reviews                0
Size                   0
Price                  0
Content_Rating         0
Last_Updated_On        0
Release_Version        0
OS_Version_Required    0
Downloads              0
dtype: int64

In [138]:
# print(df['Release_Version'].nunique())
print(df['OS_Version_Required'].nunique())
print(df['Content_Rating'].nunique())
print(df['Price'].nunique())
print(df['Size'].nunique())
print(df['Category'].nunique())
df = df[df['Downloads']!=17]
print(df['Downloads'].value_counts())
df.isnull().sum(axis=0)

27
6
83
439
51
8     3158
10    2707
6     2239
9     1360
12    1277
7     1243
4     1221
11     912
5      763
2      508
3      364
14     266
13     247
0       91
1       84
15      42
16      33
Name: Downloads, dtype: int64


Offered_By             0
Category               0
Rating                 0
Reviews                0
Size                   0
Price                  0
Content_Rating         0
Last_Updated_On        0
Release_Version        0
OS_Version_Required    0
Downloads              0
dtype: int64

In [139]:
print(test['OS_Version_Required'].nunique())
print(test['Content_Rating'].nunique())
print(test['Price'].nunique())
print(test['Size'].nunique())
print(test['Category'].nunique())

37
6
95
560
51


In [140]:
df.dtypes

Offered_By              object
Category                object
Rating                 float64
Reviews                  int64
Size                    object
Price                   object
Content_Rating          object
Last_Updated_On         object
Release_Version         object
OS_Version_Required     object
Downloads                int32
dtype: object

In [141]:
df.isnull().sum(axis=0)

Offered_By             0
Category               0
Rating                 0
Reviews                0
Size                   0
Price                  0
Content_Rating         0
Last_Updated_On        0
Release_Version        0
OS_Version_Required    0
Downloads              0
dtype: int64

# Playing with price column

In [142]:
df['Price'] = df['Price'].replace('Free',0).astype(float)
test['Price'] = test['Price'].replace('Free',0).astype(float)

In [143]:
print(df['Price'].describe())
print(test['Price'].describe())

count    16515.000000
mean        30.788512
std        573.733270
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max      29831.254200
Name: Price, dtype: float64
count    24776.000000
mean        33.380721
std        605.037697
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max      29832.000000
Name: Price, dtype: float64


# Playing with size

In [ ]:
df['size'] = df

# Playing with content rating

In [144]:
le1 = LabelEncoder()
le1.fit(df['Content_Rating'])
df['Content_Rating_encoded'] = le1.transform(df['Content_Rating'])
test['Content_Rating_encoded'] = le1.transform(test['Content_Rating'])

# Playing with category column

In [145]:
le2 = LabelEncoder()
le2.fit(df['Category'])
df['Category_encoded'] = le2.transform(df['Category'])
test['Category_encoded'] = le2.transform(test['Category'])

# Playing with last updated on

In [146]:
df['Year'] = pd.to_datetime(df['Last_Updated_On'],format='%b %d %Y').dt.year

In [147]:
df['gap'] = 2020 - df['Year']

In [162]:
test['Year'] = pd.to_datetime(test['Last_Updated_On'],format='%b %d %Y').dt.year
test['gap'] = 2020 - test['Year']

In [148]:
df.head()

,Offered_By,Category,Rating,Reviews,Size,Price,Content_Rating,Last_Updated_On,Release_Version,OS_Version_Required,Downloads,Content_Rating_encoded,Category_encoded,Year,gap
0,ps_id-24654,Finance,4.18,1481,Varies with device,0.0000,Everyone,May 05 2020,Varies with device,Varies with device,8,1,12,2020,0
1,ps_id-35329,Music And Audio,4.81,302,10M,0.0000,Everyone,Mar 26 2020,3.9.18,4.1 and up,5,1,38,2020,0
2,ps_id-11044,Game Casual,4.27,374,27M,0.0000,Everyone,May 01 2020,1.10.1,4.1 and up,6,1,21,2020,0
3,ps_id-36068,Business,4.03,122058,Varies with device,0.0000,Teen,May 02 2020,Varies with device,Varies with device,12,4,4,2020,0
4,ps_id-35831,Medical,4.60,358,Varies with device,297.5742,Everyone,Nov 29 2018,Varies with device,Varies with device,5,1,37,2018,2


In [149]:
df.columns

Index(['Offered_By', 'Category', 'Rating', 'Reviews', 'Size', 'Price',
       'Content_Rating', 'Last_Updated_On', 'Release_Version',
       'OS_Version_Required', 'Downloads', 'Content_Rating_encoded',
       'Category_encoded', 'Year', 'gap'],
      dtype='object')

In [158]:
X = df.drop(columns=['Downloads','Release_Version','Last_Updated_On','Size','Offered_By','Category','Price','OS_Version_Required','Content_Rating'])
y = df['Downloads']
X.dtypes

Rating                    float64
Reviews                     int64
Content_Rating_encoded      int32
Category_encoded            int32
Year                        int64
gap                         int64
dtype: object

In [159]:
def baseliner(X, y, cv=3, metric='neg_log_loss'):
    print("Baseliner Models\n")
    eval_dict = {}
    models = [LGBMClassifier(), XGBClassifier() 
                 ]
    print("Model Name \t |   CV")
    print("--" * 50)

    for index, model in enumerate(models, 0):
        model_name = str(model).split("(")[0]
        eval_dict[model_name] = {}

        results = cross_val_score(model, X, y, cv=cv, scoring=metric)
        eval_dict[model_name]['cv'] = results.mean()

        print("%s \t | %.4f \t" % (
            model_name[:12], eval_dict[model_name]['cv']))

In [160]:
baseliner(X,y,5)

Baseliner Models

Model Name 	 |   CV
----------------------------------------------------------------------------------------------------
LGBMClassifi 	 | -1.6841 	
XGBClassifie 	 | -1.2459 	


In [259]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8, nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='neg_log_loss',n_jobs=-1, cv=5)
gsearch1.fit(X,y)
gsearch1.best_params_,gsearch1.best_score_


[23:43:22] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




({'max_depth': 5, 'min_child_weight': 5}, -1.1716729261508916)

In [270]:
param_test2 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=gsearch1.best_params_['max_depth'],
 min_child_weight=gsearch1.best_params_['min_child_weight'], gamma=0, subsample=0.8, colsample_bytree=0.8, nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test2, scoring='neg_log_loss',n_jobs=-1, cv=5)
gsearch2.fit(X,y)
gsearch2.best_params_,gsearch2.best_score_


[23:57:19] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




({'gamma': 0.3}, -1.1715546459908666)

In [271]:
param_test3 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=gsearch1.best_params_['max_depth'],
 min_child_weight=gsearch1.best_params_['min_child_weight'], gamma=gsearch2.best_params_['gamma'], subsample=0.8, colsample_bytree=0.8, nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test3, scoring="neg_log_loss",n_jobs=-1, cv=5)
gsearch3.fit(X,y)
gsearch3.best_params_,gsearch3.best_score_

[00:08:29] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




({'colsample_bytree': 0.9, 'subsample': 0.7}, -1.1680081732409278)

In [272]:
param_test4 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100],'learning_rate':[i/10.0 for i in range(0,5)]}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=gsearch1.best_params_['max_depth'],
 min_child_weight=gsearch1.best_params_['min_child_weight'], gamma=gsearch2.best_params_['gamma'], subsample=gsearch3.best_params_['subsample'], colsample_bytree=gsearch3.best_params_['colsample_bytree'], nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test4, scoring='neg_log_loss',n_jobs=-1, cv=5)
gsearch4.fit(X,y)
gsearch4.best_params_,gsearch4.best_score_

[00:17:51] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




({'learning_rate': 0.1, 'reg_alpha': 1}, -1.1665219924579309)

In [283]:
prob = 0
for seed_val in range(100):
    print (seed_val)
    m= XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=5, gamma=0.3, subsample=0.7, colsample_bytree=0.9, njobs=-1, scale_pos_weight=1, seed=seed_val,reg_alpha=1)
    m.fit(X,y,eval_metric='mlogloss')
    predict=m.predict_proba(test.drop(columns=['Release_Version','Last_Updated_On','Size','Offered_By','Category','Price','OS_Version_Required','Content_Rating']))
    prob+=predict
prob = pd.DataFrame(prob/100)

0
[00:44:25] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { njobs, scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


1
[00:44:34] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { njobs, scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


2
[00:44:43] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { njobs, scale_pos_weight } might not be used.

  This may not be accurate due

21
[00:48:05] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { njobs, scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


22
[00:48:15] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { njobs, scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


23
[00:48:26] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { njobs, scale_pos_weight } might not be used.

  This may not be accurate 

42
[00:51:47] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { njobs, scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


43
[00:51:57] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { njobs, scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


44
[00:52:08] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { njobs, scale_pos_weight } might not be used.

  This may not be accurate 

63
[00:55:30] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { njobs, scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


64
[00:55:41] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { njobs, scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


65
[00:55:52] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { njobs, scale_pos_weight } might not be used.

  This may not be accurate 

84
[00:59:13] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { njobs, scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


85
[00:59:24] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { njobs, scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


86
[00:59:34] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { njobs, scale_pos_weight } might not be used.

  This may not be accurate 

In [260]:
prob = gsearch1.best_estimator_.predict_proba(test.drop(columns=['Release_Version','Last_Updated_On','Size','Offered_By','Category','Price','OS_Version_Required','Content_Rating']))

In [284]:

prob.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.000211,0.000197,0.000269,0.000261,0.000356,0.000893,0.157441,0.189929,0.569558,0.069033,0.009718,0.000792,0.000448,0.000267,0.000300,0.000182,0.000145
1,0.000195,0.000191,0.000211,0.000313,0.000330,0.000428,0.013736,0.061228,0.455141,0.358161,0.105094,0.002535,0.000586,0.000399,0.000723,0.000359,0.000371
2,0.000150,0.000145,0.000209,0.000197,0.000293,0.000302,0.000471,0.000648,0.000843,0.001332,0.074042,0.242190,0.672831,0.002578,0.003212,0.000383,0.000174
3,0.000282,0.000263,0.000358,0.000439,0.011601,0.029087,0.474679,0.189705,0.270653,0.017015,0.003975,0.000521,0.000430,0.000319,0.000265,0.000199,0.000207
4,0.000217,0.000223,0.000277,0.000326,0.000486,0.001623,0.095990,0.178962,0.649060,0.056109,0.014589,0.000682,0.000342,0.000303,0.000339,0.000225,0.000247


In [285]:
z = pd.DataFrame(np.zeros(shape=(24776,1)))
z.rename(columns={0:17},inplace=True)
z.fillna(0,inplace=True)
z = z.astype(np.float32)
z.shape

(24776, 1)

In [286]:
prob = prob.join(z,1)

C:\Users\ASUS\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:1092: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(


In [287]:
prob.shape

(24776, 18)

In [288]:
prob.fillna(0,inplace=True)

In [289]:
prob.tail()
prob.set_index(0,inplace=True)

In [290]:
prob.to_csv('Machinehack_15.csv')

In [248]:
s = pd.read_csv('Sample_Submission.csv')

In [232]:
s.shape

(24776, 18)

In [233]:
prob.shape

(24776, 18)

In [234]:
test.shape

(24776, 14)

In [249]:
prob.dtypes

0     float32
1     float32
2     float32
3     float32
4     float32
5     float32
6     float32
7     float32
8     float32
9     float32
10    float32
11    float32
12    float32
13    float32
14    float32
15    float32
16    float32
17    float32
dtype: object